# AugmentedGPT Playground

Please run `poetry install` under the project root directory. This will install all the necessary dependencies for you.

## Imports and set API key

In [2]:
from augmented_gpt import *
import getpass, dotenv, os

dotenv.load_dotenv()
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key: ')

## Create a GPT instance that can use tools.

In [3]:
@tool
def send_email(
    topic: str = param("The email topic"),
    to: str = param("The recipient email address"),
    content: str = param("The content of the email"),
):
    """Send an email to a recipient"""
    print(f"--- SEND EMAIL ---")
    print(f"TOPIC: {topic}")
    print(f"TO: {to}")
    print(f"CONTENT:\n\n{content}")
    print(f"---")
    return "done."

gpt = AugmentedGPT(tools=[send_email, plugins.CalculatorPlugin()], debug=True)

completion = gpt.chat_completion([
    Message(role=Role.USER, content="Send Steve an email and tell him the distance between moon and earth divided by two."),
])

async for response in completion:
    print(response)

Message(role=<Role.ASSISTANT: 'assistant'>, content=None, name=None, function_call=None, tool_calls=[ToolCall(id='call_UYgsoLuYBrevw2Mo58QNPoRH', function=FunctionCall(name='ToolRegistry-evaluate', arguments={'expression': '384400 / 2'}), type='function')], tool_call_id=None)
Message(role=<Role.TOOL: 'tool'>, content='192200.0', name=None, function_call=None, tool_calls=[], tool_call_id='call_UYgsoLuYBrevw2Mo58QNPoRH')
Message(role=<Role.ASSISTANT: 'assistant'>, content=None, name=None, function_call=None, tool_calls=[ToolCall(id='call_9G53xbu8mUiBWoQNW2CYaEqz', function=FunctionCall(name='send_email', arguments={'topic': 'Distance Between Moon and Earth', 'to': 'steve@example.com', 'content': 'Dear Steve,\n\nThe distance between the Moon and Earth divided by two is approximately 192,200 kilometers.\n\nBest regards.'}), type='function')], tool_call_id=None)
--- SEND EMAIL ---
TOPIC: Distance Between Moon and Earth
TO: steve@example.com
CONTENT:

Dear Steve,

The distance between the Mo